In [1]:
import pyterrier as pt
import pandas as pd

In [101]:
!pwd

/users/tr.xiaow/q2q-queries


In [2]:
import pandas as pd
import re
def processQuery(query):
    query = re.sub(r"[^a-zA-Z0-9¿]+", " ", query)
    return query

def Q2(inputDF):
    inputDF['query']=inputDF['query'].apply(lambda query: processQuery(query))
    return inputDF

In [3]:
#provide 8GB RAM to Terrier's JVM
pt.init(mem=8000)

In [78]:
import pandas as pd

# indexloc="/local/terrier/Indices/msMarco/blockFormattedIndexing/data.properties"
indexloc="/local/terrier/Indices/msMarco/blockFormattedIndexingStem/data.properties"
# indexloc="/users/tr.xiaow/MsmarcoNostemming/data.properties"

qrelsFile="/local/terrier7/Collections/msMarco/DeepLearning19/Document_Ranking/qrels-docs.txt"
qrelsTest = pt.Utils.parse_qrels(qrelsFile)


topicsFile="/local/terrier7/Collections/msMarco/DeepLearning19/Document_Ranking/msmarco-test2019-queries.tsv.gz"
topicsTest=pd.read_csv(topicsFile, sep='\t', names=['qid', 'query'])
topicsTest=Q2(topicsTest)
topicsTest["qid"]=topicsTest["qid"].astype(str)
# drop queries without relevant documents
topicsTest = topicsTest.merge(qrelsTest, on="qid")[["qid", "query"]].drop_duplicates()


# TODO we need a singlelinetrec query API
# topicsTest["query"] = topicsTest["query"].apply(lambda x : x.replace('""', "").replace("'", "").replace("_", "").replace(".", "").replace("?", ""))


In [79]:
predPath="/users/tr.xiaow/q2q-queries/q_catseq_ML43.txt"
topicsPred=pd.read_csv(predPath,sep='\t',names=['qid','query'])
topicsPred["query"]=topicsPred["query"].astype(str)
topicsPred=Q2(topicsPred)
topicsPred["qid"]=topicsPred["qid"].astype(str)

In [10]:
DPH_br = pt.BatchRetrieve(indexloc, controls={"wmodel" : "DPH"}, verbose=True)
BM25_br= pt.BatchRetrieve(indexloc, controls={"wmodel" : "BM25"}, verbose=True)
DPH_br_qe = pt.BatchRetrieve(indexloc, controls={"wmodel" : "DPH", "qe" : "on"}, verbose=True)

00:16:15 WARN  [main] - [CompressingMetaIndex]- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 4.5 GiB of memory would be required.
00:16:16 WARN  [main] - [CompressingMetaIndex]- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 4.5 GiB of memory would be required.
00:16:16 WARN  [main] - [CompressingMetaIndex]- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 4.5 GiB of memory would be required.


In [32]:
#this tests on a single topic. TODO: update as necessary, DPH_bert
evalTest = pt.pipelines.Experiment(topicsTest,[BM25_br],['map'],qrelsTest, perquery=True)
#                         names=[ "BM25"])[["map"]]
evalTest

100%|██████████| 43/43 [00:24<00:00,  1.78q/s]


,156493,1110199,1063750,130510,489204,573724,1133167,527433,1037798,915593,...,87452,855410,1114646,19335,47923,405717,182539,1113437,1129237,146187
BR(BM25),{'map': 0.7380501454656035},{'map': 0.14677535844094677},{'map': 0.027881839299792294},{'map': 0.5132425435582879},{'map': 0.04088780966346987},{'map': 0.37630640775024754},{'map': 0.576761976257666},{'map': 0.20342597908114746},{'map': 0.011752640069903937},{'map': 0.4091642490934538},...,{'map': 0.20390562670042695},{'map': 1.0},{'map': 0.3188636597335874},{'map': 0.279452316662596},{'map': 0.4385126095530747},{'map': 0.4231551010452408},{'map': 0.3821108730070625},{'map': 0.2209210349577537},{'map': 0.1825246043806175},{'map': 0.20047006359971775}


In [31]:
# evalTest

In [12]:
evalPred = pt.pipelines.Experiment(topicsPred,[BM25_br],['map'],qrelsTest, perquery=True)
#                         names=[ "BM25"])[["map"]]
evalPred

100%|██████████| 43/43 [00:27<00:00,  1.56q/s]


,1037798,104861,1063750,1103812,1106007,1110199,1112341,1113437,1114646,1114819,...,489204,490595,527433,573724,833860,855410,87181,87452,915593,962179
BR(BM25),{'map': 0.012693775826382502},{'map': 0.09026731193989111},{'map': 0.01698299245044184},{'map': 0.06067052337428169},{'map': 0.026861764208745954},{'map': 0.11348574098517998},{'map': 0.15477386785951977},{'map': 0.34495376456668037},{'map': 0.3188636597335874},{'map': 0.46237040573197197},...,{'map': 0.017324764290027654},{'map': 0.5520464177870904},{'map': 0.06181127052080511},{'map': 0.0005130646457037541},{'map': 0.15045274143620396},{'map': 1.0},{'map': 0.22509927303577523},{'map': 0.07835673760291148},{'map': 0.4198045719820479},{'map': 0.7338423952628346}


In [71]:
topicsTest=topicsTest.reset_index()
topicsPred=topicsPred.reset_index()
topicsMerge=topicsTest.merge(topicsPred,on='qid',how='left' )
topicsMerge=topicsMerge.drop(['index_x','index_y','level_0'], axis=1)

In [28]:
def eval_process(eval):
    evalFile=eval.iloc[0]
    evalFile.to_json("evalFile.json")
    evalFile=pd.read_json ("evalFile.json")
    evalFile=evalFile.stack()
    evalFileDF=pd.DataFrame(evalFile, columns=['map']).reset_index(drop=True)
    return evalFileDF


In [67]:
evalTest=eval_process(evalTest)
# evalTest

In [22]:
evalPred=eval_process(evalPred)
# evalPred

In [94]:
eval_merge = evalTest.merge(evalPred, left_index=True, right_index=True)
eval_merge['difference(Pred-Test)'] = eval_merge['map_y'] - eval_merge['map_x']
eval_merge=eval_merge.rename(columns={'map_x':'map_test','map_y':'map_Pred'})

In [100]:
# topicsTest.reset_index()
eval_M = topicsMerge.merge(eval_merge, left_index=True, right_index=True,how='left')
# eval_M = eval_M.merge(topicsPred,left_index=True, right_index=True,how='')
eval_M.to_csv('/users/tr.xiaow/perqueryJup/catseq_Ipv.txt',sep='\t',index=False,header=0)

In [97]:
improvedQ = eval_M[(eval_M['difference(Pred-Test)']> 0)]
print("improved query number: ",improvedQ.shape[0])
improvedQ

improved query number:  17


,qid,query_x,query_y,map_test,map_Pred,difference(Pred-Test)
7,527433,types of dysarthria from cerebral palsy,types of dysarthria from cerebral palsy what c...,0.203426,0.344954,0.141528
8,1037798,who is robert gray,who is robert gray what is robert gray,0.011753,0.318864,0.307111
9,915593,what types of food can you cook sous vide,what types of food can you cook sous vide what...,0.409164,0.462370,0.053206
13,1117099,what is a active margin,what is a active margin active active margin d...,0.058815,0.907150,0.848335
15,451602,medicare s definition of mechanical ventilation,medicare s definition of mechanical ventilatio...,0.034219,0.215918,0.181699
16,359349,how to find the midsegment of a trapezoid,how to find the midsegment of a trapezoid what...,0.516503,0.576762,0.060259
17,1115776,what is an aml surveillance analyst,what is an aml surveillance analyst what is am...,0.504738,0.513243,0.008504
18,1112341,what is the daily life of thai people,what is the daily life of thai people what is ...,0.087100,0.441470,0.354370
20,104861,cost of interior concrete flooring,cost of interior concrete flooring cost of con...,0.180883,0.450156,0.269273
21,833860,what is the most popular food in switzerland,what is the most popular food in switzerland w...,0.150453,0.575062,0.424609


In [98]:
improvedQ = eval_M[(eval_M['difference(Pred-Test)']< 0)]
print("Decreased query number: ",improvedQ.shape[0])
improvedQ

Decreased query number:  26


,qid,query_x,query_y,map_test,map_Pred,difference(Pred-Test)
0,156493,do goldfish grow,do goldfish grow how many baby water in a gold...,0.738050,0.012694,-0.725356
1,1110199,what is wifi vs bluetooth,what is wifi vs bluetooth what is a wifi,0.146775,0.090267,-0.056508
2,1063750,why did the us volunterilay enter ww1,why did the us volunterilay enter ww1 when did...,0.027882,0.016983,-0.010899
3,130510,definition declaratory judgment,definition declaratory judgment declaratory ju...,0.513243,0.060671,-0.452572
4,489204,right pelvic pain causes,right pelvic pain causes what causes pelvic re...,0.040888,0.026862,-0.014026
5,573724,what are the social determinants of health,what are the social determinants of health wha...,0.376306,0.113486,-0.262821
6,1133167,how is the weather in jamaica,how is the weather in jamaica how is the weath...,0.576762,0.154774,-0.421988
10,264014,how long is life cycle of flea,how long is life cycle of flea how long does i...,0.576607,0.529809,-0.046798
11,1121402,what can contour plowing reduce,what can contour plowing reduce what is contou...,0.563966,0.030019,-0.533947
12,962179,when was the salvation army founded,when was the salvation army founded what is th...,0.518441,0.407053,-0.111387
